In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install transformers datasets torchaudio --quiet


# Loaded the whisper model for the trancript generation.


In [3]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install -y ffmpeg


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-fcd_ze97
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-fcd_ze97
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.6 MB/s eta 0:00:000:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803707 sha256=9fd093b9ace15b4634225ee6da13ea4ad7d23e7d45109c57268ff18fd355d518
  Stored in directory: /tmp/pip-ephem-wheel-cache-e7hzb85t/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/lin

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torchaudio
import os


# Loaded the finetuned deberta model 
The deberta large model was finetuned on the provided dataset by shl.

In [7]:
model_path = "/kaggle/input/deberta-v3-large-700epoch3best"  # Replace with your path
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNo

# used whisper for trancribing the input audio


In [4]:
import whisper

# Load Whisper model (small or medium, depending on GPU)
asr_model = whisper.load_model("medium")  # or "small" if GPU is limited

# Transcribe audio
def transcribe(audio_path):
    result = asr_model.transcribe(audio_path)
    return result["text"]


100%|█████████████████████████████████████| 1.42G/1.42G [00:22<00:00, 69.0MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

# used the finetunned model for label prediction

In [18]:
id_to_label = {
    0: '1.0',
    1: '1.5',
    2: '2.0',
    3: '2.5',
    4: '3.0',
    5: '3.5',
    6: '4.0',
    7: '4.5',
    8: '5.0'
}

def predict_label(text):
    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predicted_class = torch.argmax(probs, dim=1).item()
        mapped_label = id_to_label.get(predicted_class, "invalid audio")
        return mapped_label, probs.tolist()
    except Exception as e:
        return "invalid audio", []


# full pipeline

In [21]:
def audio_to_label(audio_path):
    transcription = transcribe(audio_path)
    print("Transcription:", transcription)

    label, probs = predict_label(transcription)
    print("Predicted Label:", label)
    print("Confidence Scores:", probs)
    return label


# test case


In [22]:
audio_path = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/audio_1008.wav"
predicted_label = audio_to_label(audio_path)


Transcription:  The school playground is lively and filled with colorful equipment like swings and slides. Children engage in various games like dodgeball or soccer. It's normally filled with laughter and chatters in the air. It's normally accompanied with sounds of swings, squeaking and balls bouncing. It's a dynamic environment where friendships are formed and memories are made. Those are experiences from the school playground and scenes you could find on the school playground.
Predicted Label: 4.0
Confidence Scores: [[0.011450857855379581, 0.010414010845124722, 0.0378461591899395, 0.026868218556046486, 0.08004546910524368, 0.020012935623526573, 0.3226325511932373, 0.19212931394577026, 0.29860052466392517]]
